### Personal Finance Manager
 - This python program will help to check the expenses and incomes from different accoutns and keep track of the finances
## What can this programme do?
 - Create new account
 - Delete account
 - Save incoming amount
 - Save expenses
 - Produce a report on expenses v income
### Further production 
 - Categorize expenses
 - Automate producing reports 

## Importing libraries


In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import json

## Class : Account
This class manages the details of a single account, including its balance and transactions.


In [56]:
class Account:
    def __init__(self, name, initial_balance = 0):
        self.name = name
        self.balance = initial_balance
        self.transactions = []
        
    def add_transaction(self, transaction):
        self.transactions.append(transaction)
        self.balance += transaction.amount

    def get_balance(self):
        return self.balance

    

## Class : Transaction
This class handles individual transactions (income, expenses)

- Initialize a new transaction.
        
    - param date: The date of the transaction (e.g., '2024-09-01')
    - param description: A description of the transaction (e.g., 'Groceries')
    - param amount: The amount of the transaction (negative for expense, positive for income)
    - param category: Optional category for the transaction (e.g., 'Food', 'Salary')


In [53]:
class Transaction:
    def __init__(self,date, description, amount, category=None):
        self.date = date
        self.description = description
        self.amount = amount
        self.category = category
          
    def __str__(self):
        category = self.category if self.category else "Uncategorised"
        return f"{self.date}: {self.description} - {self.amount} ({category})"
        
    

## Class: Finance Tracker
Key Features for the FinanceTracker Class:
- Attributes:
    - A list or dictionary to store all accounts (or users, depending on your project scope).
    - The ability to add new accounts and transactions.
- Methods:
    - Add a new account.
    - Add a transaction to an account.
    - View balance for a specific account.


In [100]:
class FinanceTracker:
    def __init__(self, data_file='finance_data.json'):
        self.accounts = {}
        self.data_file = data_file
        self.load_data() #Load data in the start of the program

    def add_account(self, account_name, initial_balance=0):
        if account_name in self.accounts:
            print(f"Account '{account.name}' already exists")
        else:
            account = Account(account_name, initial_balance)
            self.accounts[account_name] = account
            print(f"Account '{account_name}' added.")
            self.save_data()
            
    def check_account_exists(self, account_name):
        if account_name in self.accounts:
            return True
        else:
            return False

    def show_all_accounts(self):
        """Display all accounts with their balance."""
        if not self.accounts:
            print("No accounts found.")
        else:
            for account_name, account in self.accounts.items():
                print(f"Account: {account_name}, Balance: {account.balance}")
            


    def add_transaction(self,account_name,date,description, amount, category=None):
        if account_name in self.accounts:
            transaction = Transaction(date,description,amount,category)
            self.accounts[account_name].add_transaction(transaction)
            print(f"Transaction added to '{account_name}' account.")
            self.save_data()
        else:
            print(f"Account '{account_name}' does not exist. Please create account before adding transaction.")

    def get_account_balance(self, account_name):
        if account_name in self.accounts:
            return self.accounts[account_name].get_balance()
        else:
            print(f"Account '{account_name}' does not exist.")
            return None

    def generate_report(self):
        print("Finance Summary of All accounts")
        print("-" * 30)
        total_balance = 0
        print(self.accounts)
        for account in self.accounts.values():
            balance = account.balance
            total_balance += balance
            print(f"Account: {account.name} | Balance: {balance}")
        print(f"Total Balance across all accounts: {total_balance}")
        print("-" * 30)
        
    def save_data(self):
        """Save all account and transaction data to a JSON file."""
        data = {}

        for account_name, account in self.accounts.items():
            data[account_name] = {
                'balance': account.balance,
                'transactions': [
                    {
                        'date': t.date,
                        'description': t.description,
                        'amount': t.amount,
                        'category': t.category
                    }
                    for t in account.transactions
                ]
            }
        
        # Save the data to the file
        with open(self.data_file, 'w') as f:
            json.dump(data, f, indent=4)      

    def load_data(self):
            """Load account and transaction data from a JSON file if it exists."""
            try:
                with open(self.data_file, 'r') as f:
                    data = json.load(f)
                
                # Reconstruct the accounts and transactions from the JSON data
                for account_name, account_data in data.items():
                    account = Account(account_name, account_data['balance'])
                    for transaction_data in account_data['transactions']:
                        transaction = Transaction(
                            transaction_data['date'],
                            transaction_data['description'],
                            transaction_data['amount'],
                            transaction_data.get('category')
                        )
                        account.add_transaction(transaction)
                    self.accounts[account_name] = account
                print(f"Loaded data from {self.data_file}")
            except FileNotFoundError:
                print(f"No data file found. Starting fresh.")
            except json.JSONDecodeError:
                print(f"Error loading JSON data. Starting fresh.")
            

In [101]:
def display_menu():
    print("\n--- Finance Tracker Menu ---")
    print("1. Add new account")
    print("2. Add transaction")
    print("3. View account balance")
    print("4. Generate report for all accounts")
    print("5. View category breakdown for an account")
    print("6. Show All accounts")
    print("7. Exit")

    

In [106]:
def main():
    tracker = FinanceTracker()

    while True:
        display_menu()
        choice = input("Select an option (1-6): ")

        if choice == "1":
            account_name = input("Enter account name: ")
            initial_balance = float(input("Enter initial balance: "))
            tracker.add_account(account_name, initial_balance)

        elif choice == "2":
            account_name = input("Enter account name: ")
            if tracker.check_account_exists(account_name):
                date = input("Enter transaction date (YYYY-MM-DD): ")
                description = input("Enter transaction description: ")
                amount = float(input("Enter transaction amount (positive for income, negative for expense): "))
                category = input("Enter transaction category (optional): ")
                tracker.add_transaction(account_name, date, description, amount, category)
            else:
                print(f"Account '{account_name}' does not exist. Please create the account first.")


        elif choice == "3":
            account_name = input("Enter account name: ")
            balance = tracker.get_account_balance(account_name)
            if balance is not None:
                print(f"Balance for {account_name}: {balance}")

        elif choice == "4":
            tracker.generate_report()

        elif choice == "5":
            account_name = input("Enter account name: ")
            tracker.category_report(account_name)

        elif choice == "6":
            tracker.show_all_accounts()

        elif choice == "7":
            print("Exiting the Finance Tracker. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


In [ ]:
# Run the interactive program
if __name__ == "__main__":
    main()

Loaded data from finance_data.json

--- Finance Tracker Menu ---
1. Add new account
2. Add transaction
3. View account balance
4. Generate report for all accounts
5. View category breakdown for an account
6. Show All accounts
7. Exit


Select an option (1-6):  6


Account: home, Balance: 785.0
Account: office, Balance: 5000.0

--- Finance Tracker Menu ---
1. Add new account
2. Add transaction
3. View account balance
4. Generate report for all accounts
5. View category breakdown for an account
6. Show All accounts
7. Exit


In [ ]:
tracker = FinanceTracker()

In [16]:
tracker.add_account("test", 600)

Account 'test' added.


In [18]:
tracker.get_balance("test")

600